In [1]:
import collections
import functools
import json
import os
import pickle
import glob
import re

import numpy as np
import tensorflow.compat.v1 as tf

In [2]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [3]:
import os
import numpy as np
import pickle
import math

def rotate(p, quat):
    R = np.zeros((3, 3))
    a, b, c, d = quat[3], quat[0], quat[1], quat[2]
    R[0, 0] = a**2 + b**2 - c**2 - d**2
    R[0, 1] = 2 * b * c - 2 * a * d
    R[0, 2] = 2 * b * d + 2 * a * c
    R[1, 0] = 2 * b * c + 2 * a * d
    R[1, 1] = a**2 - b**2 + c**2 - d**2
    R[1, 2] = 2 * c * d - 2 * a * b
    R[2, 0] = 2 * b * d - 2 * a * c
    R[2, 1] = 2 * c * d + 2 * a * b
    R[2, 2] = a**2 - b**2 - c**2 + d**2

    return np.dot(R, p)


# copy of vispy_utils.py particlify_box()
def particlify_box(center, half_edge, quat):
    
    pos = []

    # initial spacing
    offset_height = 0.02
    offset_width1 = 0.02
    offset_width2 = 0.02
    
    half_width1 = half_edge[0]
    half_height = half_edge[1]
    half_width2 = half_edge[2]

    particle_count_height = math.ceil(half_height * 2 / offset_height)
    particle_count_width1 = math.ceil(half_width1 * 2 / offset_width1)
    particle_count_width2 = math.ceil(half_width2 * 2 / offset_width2)

    offset_height = half_height * 2 / particle_count_height
    offset_width1 = half_width1 * 2 / particle_count_width1
    offset_width2 = half_width2 * 2 / particle_count_width2


    local_bottom_corner_pos = np.array([-half_width1, -half_height, - half_width2])


    for h in range(0, particle_count_height + 1):
        for w in range(0, particle_count_width1):
            pos.append(local_bottom_corner_pos + np.array([offset_width1 * w, offset_height * h, 0]))
        for w in range(0, particle_count_width2):
            pos.append(local_bottom_corner_pos + np.array([half_width1 * 2, offset_height * h, offset_width2 * w]))
        for w in range(0, particle_count_width1):
            pos.append(local_bottom_corner_pos + np.array([half_width1 * 2 - offset_width2 * w, offset_height * h, half_width2 * 2]))
        for w in range(0, particle_count_width2):
            pos.append(local_bottom_corner_pos + np.array([0, offset_height * h, half_width2 * 2 - offset_width2 * w]))

    for r in range(1, particle_count_width1):
        for c in range(1, particle_count_width2):
            pos.append(local_bottom_corner_pos + np.array([offset_width1 * r, half_height * 2, offset_width2 * c]))
            pos.append(local_bottom_corner_pos + np.array([offset_width1 * r, 0, offset_width2 * c]))
        

    pos = np.asarray(pos, dtype=np.float64)
    
    for i in range(len(pos)):
        pos[i] = rotate(pos[i], quat)

    pos[:,0] += center[0]
    pos[:,1] += center[1]
    pos[:,2] += center[2]
        
    # pos = np.concatenate((pos, np.ones([len(pos), 1])), 1)
    
    return pos

def add_grips(positions, shape_states, half_edge, restpos=False):
    pos_all = []
    for r in range(len(positions)):
        pos_grip_iter = []
        for i in range(len(positions[0])):

            pos_grips = []

            for i_grip in range(len(shape_states[r, i])):

                pos = shape_states[r, i][i_grip][0:3]
                quat = shape_states[r, i][i_grip][6:10]
                pos_grip = particlify_box(pos, half_edge[i_grip], quat)

                if restpos: pos_grips.append(np.concatenate([pos_grip, pos_grip], axis=1))
                else : pos_grips.append(pos_grip)

            pos_grips = np.vstack(pos_grips)
            pos_grips = pos_grips.reshape(-1, pos_grips.shape[-1])


            pos_grip_iter.append(np.concatenate((positions[r,i], pos_grips), 0))

        pos_all.append(pos_grip_iter)

    pos_all = np.asarray(pos_all, dtype=np.float64)

    return pos_all

In [12]:
DATA_PATICLE_PATH = "/home/issei/Documents/UCSD/SuLab/Neural_Simulation/tmp/Finetune/RiceGrip/FLEX/finetune/raw"
DATA_DEPTH_PATH = "/home/issei/Documents/UCSD/SuLab/Neural_Simulation/tmp/Finetune/RiceGrip/MPM/finetune/depth"
OUT_PATH = "/home/issei/Documents/UCSD/SuLab/Neural_Simulation/tmp/Finetune/RiceGrip/FLEX/finetune/data"

os.makedirs(OUT_PATH, exist_ok=True)
# WRITER_NAME = os.path.join(WRITER_DIR, "train.tfrecord")
has_context = True
has_restpos = True
is_mpm = False

generate_tfrecord(DATA_PATICLE_PATH, DATA_DEPTH_PATH, os.path.join(OUT_PATH, "train.tfrecord"), 0, 8, has_context, has_restpos)

In [11]:
def generate_tfrecord(data_particle_path, data_depth_path, writer_name, idx_start, idx_end, _HAS_CONTEXT=True, _HAS_RESTPOS=True):

    writer = tf.python_io.TFRecordWriter(writer_name)
    
    files_particle = glob.glob(os.path.join(data_particle_path, "*.npy"))
    files_depths = glob.glob(os.path.join(data_depth_path, "*/*.npy"))
    
    for file_i in range(idx_start, idx_end):
#         file_particle = os.path.join(data_particle_path, "{:0>4}.npy".format(str(file_i)))
#         file_depth = os.path.join(data_depth_path, "{:0>4}.npy".format(str(file_i)))

        file_particle = files_particle[0]
        file_depth = files_depths[file_i]

        
        
        d = np.load(file_particle, allow_pickle=True).item()
        d_depth = np.load(file_depth, allow_pickle=True)
        
        # print(d_particle)
#         print(d_depth.shape)
#         print(d_depth.mean())
        
        print(f'{file_i}', end="\r",)
        

        
        d['new_positions'] = add_grips(np.concatenate([d['positions'][:,:,:,:3], d['positions'][:,:,:,:3]], axis=3) , d['shape_states'], d['scene_info'], _HAS_RESTPOS)
        
        n_particle_plasticine = len(d['positions'][0,0])
        
        positions = d['new_positions'][0].astype(np.float32)
        
        # print(positions[0].min(axis=0), positions[0].max(axis=0))
        step_contexts = []
        for _ in range(0, len(positions)):
            if is_mpm:
                step_contexts.append(d['YS'])
                step_contexts.append(d['E'])
                step_contexts.append(d['nu'])
            else:
                step_contexts.append(d['clusterStiffness'])
                step_contexts.append(d['clusterPlasticThreshold'])
                step_contexts.append(d['clusterPlasticCreep'])

        positions = np.asarray(positions)
        step_contexts = np.asarray(step_contexts)

        # Create feature list
        positions_bytes_list = []
        for pos in positions: # per frame
            positions_bytes = pos.tobytes()
            positions_bytes = tf.train.Feature(bytes_list = tf.train.BytesList(value=[positions_bytes]))
            positions_bytes_list.append(positions_bytes)
        
        depth_bytes_list = []
        for dep in d_depth: # per frame
            depth_bytes = dep.tobytes()
            depth_bytes = tf.train.Feature(bytes_list = tf.train.BytesList(value=[depth_bytes]))
            depth_bytes_list.append(depth_bytes)

        step_context_bytes_list = []
        for step_context in step_contexts: # per frame
            step_context_bytes = np.float32(step_context).tobytes()
            step_context_bytes = tf.train.Feature(bytes_list = tf.train.BytesList(value=[step_context_bytes]))
            step_context_bytes_list.append(step_context_bytes)

        positions_feature_list = tf.train.FeatureList(feature=positions_bytes_list)
        depth_bytes_list = tf.train.FeatureList(feature=depth_bytes_list)
        if _HAS_CONTEXT:
            step_context_feature_list = tf.train.FeatureList(feature=step_context_bytes_list)

        particle_type = np.ones([positions[0].shape[0]], dtype=np.int64)
        particle_type[n_particle_plasticine:] += 2
        particle_type = particle_type.tobytes()
        particle_type_feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[particle_type]))

        key = np.int64(file_i)
        key_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[key]))

        sequence_dict = {'position': positions_feature_list, 'step_context': step_context_feature_list, 'depths': depth_bytes_list}

        context_dict = {'key': key_feature, 'particle_type': particle_type_feature}

        sequence_context = tf.train.Features(feature=context_dict)
        # now create a list of feature lists contained within dictionary
        sequence_list = tf.train.FeatureLists(feature_list=sequence_dict)

        example = tf.train.SequenceExample(context=sequence_context, feature_lists=sequence_list)

        writer.write(example.SerializeToString())

        file_i += 1

    writer.close()